In [1]:
from keras.layers import Concatenate, Activation, Lambda, Subtract, Dot, Multiply
from keras.layers import Input, Dense, Flatten, Convolution1D, RepeatVector, Add
from keras.models import Model, Sequential
from keras.initializers import Zeros
from keras.applications.vgg19 import VGG19
from keras.activations import softmax
from keras import backend as K
import sys, glob, math, os
import cv2
import numpy as np
import re

ncluster = 50
nfeature = 196
nfeature_dim = 512
nbatch = 8

Using TensorFlow backend.


In [2]:
def custom_softmax(x):
    y = softmax(x, axis=1)
    return y

In [3]:
def transpose(x):
    x = K.permute_dimensions(x,(1,2,0))
    y = K.transpose(x)
    return y

In [4]:
def make_feature_tensor(x,i,c):
    # make [xi-c(1) xi-c(2) ... xi-c(n)] tensor
    t = x[:,i,:]
    z = K.repeat(t, ncluster)
    z = z - c
#    z = K.permute_dimensions(z,(0,1,2))
    return z

In [5]:
def rep_tensor_and_mult(w,x,c,batch_size):
    zz = K.zeros(shape=(batch_size,nfeature_dim,ncluster))
    
    for i in range(0,nfeature):
        # Feature vector
        t = x[:,i,:]
        t = K.repeat(t, ncluster)
        t = t - c
        t = K.permute_dimensions(t,(1,2,0))
        t = K.transpose(t)
        
        # Weight vector
        ww = w[:,i,:]
        z = K.repeat(ww, nfeature_dim)
                
        tmp = z*t
        zz = zz + tmp
        # K.update_add(zz,tmp)
        #zz = zz + tmp
        #zz = K.update_add(zz,tmp)
    
    return zz

In [6]:
def L1_normalize(x):
    #x = K.l2_normalize(x, axis=1)
    x = x / K.sum(x, axis=1, keepdims=True)
    return x

In [7]:
data = np.load("data.npy")
center = np.load("centers.npy")
centers = np.zeros((data.shape[0],center.shape[0],center.shape[1]))
for i in range(0,data.shape[0]):
    centers[i,:,:]=center

me = np.load("mean.npy")
stv = np.load("stv.npy")

In [8]:
def netvlad_model():
    # input feature and cluster centers
    x_input = Input([nfeature,nfeature_dim], name='x_input')
    c_input = Input([ncluster,nfeature_dim], name='c_input')
        
    conv_1 = []
        
    for i in range(0,ncluster):
        x = Convolution1D(1, 1, padding='same', use_bias=True)(x_input)
        conv_1.append(x)
    
    w = Concatenate(axis=2)(conv_1)
    w = Lambda(custom_softmax, name='cumstom_softmax')(w)
    z = Lambda(rep_tensor_and_mult, arguments={'x': x_input,'c': c_input,'batch_size': nbatch}, name='rep_tensor_mult')(w)
    z = Flatten()(z)
    z = Lambda(L1_normalize)(z)
    
    model = Model(inputs=[x_input,c_input], outputs=z)
    
    model.summary()
 
    return model
#END vgg19_lower_model()

In [9]:
print(data.shape)
print(centers.shape)
print(me.shape)
print(stv.shape)

(184, 196, 512)
(184, 50, 512)
(512,)
(512,)


In [10]:
model = netvlad_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
x_input (InputLayer)             (None, 196, 512)      0                                            
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 196, 1)        513         x_input[0][0]                    
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 196, 1)        513         x_input[0][0]                    
____________________________________________________________________________________________________
conv1d_3 (Conv1D)                (None, 196, 1)        513         x_input[0][0]                    
___________________________________________________________________________________________

In [11]:
pred = model.predict([data,centers], batch_size=nbatch, verbose=1)

184/184 [==============================] - 4s     


In [12]:
print(pred.shape)
print(pred[0,:])

(184, 25600)
[  1.14085606e-06   1.35091136e-06   8.45598549e-07 ...,  -8.43863745e-07
   9.57737029e-07  -8.31367015e-06]


In [13]:
#from keras.utils import plot_model
#
#plot_model(model, to_file='model.png')

In [13]:
print(pred[0,:])
print(sum(pred[100,:]))

[  1.14085606e-06   1.35091136e-06   8.45598549e-07 ...,  -8.43863745e-07
   9.57737029e-07  -8.31367015e-06]
1.00000010941
